# Read me


## please run the Exploratory and Association Rules Analysis first. This code requires pickles from the Association Rule Mining.

In [ ]:
pip install scikit-posthocs

In [ ]:
import pickle
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import ticker
import joypy
import os
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler
from factor_analyzer.factor_analyzer import calculate_kmo
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import r2_score
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from math import ceil
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import networkx as nx
import matplotlib.colors as mcolors
from itertools import product
import pandas as pd
from matplotlib.colors import ListedColormap
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
from sklearn.metrics import pairwise_distances
from sklearn.manifold import MDS
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
import re
import statsmodels.api as sm
from scipy.cluster.hierarchy import linkage as sch_linkage
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
from collections import defaultdict
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import warnings
import scikit_posthocs as sp

In [ ]:
# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load the dictionary from the file
with open('meta_dfs.pkl', 'rb') as file:
    meta_dfs = pickle.load(file)

In [ ]:
# Load the dictionary from the file
with open('meta_dfs_redundant.pkl', 'rb') as file:
    meta_dfs_redundant = pickle.load(file)

In [ ]:
sales_data = pd.read_csv('anon_sales.csv')
sales_data.describe().T

In [ ]:
# Define the directory and file path
directory = os.path.expanduser("~/Desktop/Master Project newest/data")
file_path = os.path.join(directory, "combined_scores.csv")

# Read the CSV file
best_rules = pd.read_csv(file_path)

In [ ]:
# Replace 'sub_family_' with '' (empty string) to leave only the number part
sales_data['family'] = sales_data['family'].str.replace('family_', '')

# Optionally, if you want to convert the column to numeric type
sales_data['family'] = pd.to_numeric(sales_data['family'])

In [ ]:
# Replace 'sub_family_' with '' (empty string) to leave only the number part
sales_data['sub_family'] = sales_data['sub_family'].str.replace('sub_family_', '')

# Optionally, if you want to convert the column to numeric type
sales_data['sub_family'] = pd.to_numeric(sales_data['sub_family'])

In [ ]:
# Replace 'sub_family_' with '' (empty string) to leave only the number part
sales_data['shopID'] = sales_data['shopID'].str.replace('ShopID_', '')

# Optionally, if you want to convert the column to numeric type
sales_data['shopID'] = pd.to_numeric(sales_data['shopID'])

In [ ]:
# Replace 'sub_family_' with '' (empty string) to leave only the number part
sales_data['product_code'] = sales_data['product_code'].str.replace('Product_', '')

In [ ]:
# Replace 'trx_id' with '' (empty string) to leave only the number part
sales_data['trx_id'] = sales_data['trx_id'].str.replace('trxID_', '')

# Optionally, if you want to convert the column to numeric type
sales_data['trx_id'] = pd.to_numeric(sales_data['trx_id'])

In [ ]:
# Convert the 'trx_date' column to datetime format
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

In [ ]:
sales_data['unit_price'] = pd.to_numeric(sales_data['unit_price'].str.replace(',', '.'), errors='coerce')

In [ ]:
# Create the new variable total_value as the product of quantity and unit_price
sales_data['total_value'] = sales_data['quantity'] * sales_data['unit_price']

In [ ]:
# Mapping from original years to new years
year_mapping = {
    1992: 2020,
    1993: 2021,
    1994: 2022,
    1995: 2023,
    1996: 2024
}

# Apply the mapping to change the year, keeping month and day the same
sales_data['trx_date'] = sales_data['trx_date'].apply(
    lambda x: x.replace(year=year_mapping[x.year])
)

sales_data['trx_date']

In [ ]:
# Create a 'year' column based on 'trx_date'
sales_data['year'] = sales_data['trx_date'].dt.to_period('Y')

# Create a 'quarter' column based on 'trx_date'
sales_data['quarter'] = sales_data['trx_date'].dt.to_period('Q')

In [ ]:
# Create 'year_integer' column by extracting year and converting to integer
sales_data['year_integer'] = sales_data['trx_date'].dt.year


In [ ]:
# Determine season
def get_season(month):
    seasons = {'winter': (12, 1, 2), 'spring': (3, 4, 5), 'summer': (6, 7, 8), 'autumn': (9, 10, 11)}
    for season, months in seasons.items():
        if month in months:
            return season
    return None

sales_data['season'] = sales_data['trx_date'].dt.month.apply(get_season)

In [ ]:
# Day of the week (Monday=1, Tuesday=2, ...)
sales_data['day_of_week_number'] = sales_data['trx_date'].dt.dayofweek + 1

In [ ]:
# Day of the week name
sales_data['day_of_week_name'] = sales_data['trx_date'].dt.day_name()

In [ ]:
sales_data['year_month'] = sales_data['trx_date'].dt.to_period('M')

# Hypothesis 1

## There are significant variations in profitability across seasons.

In [ ]:

# Extract data for each season
winter_data = sales_data[sales_data['season'] == 'winter']['total_value']
spring_data = sales_data[sales_data['season'] == 'spring']['total_value']
summer_data = sales_data[sales_data['season'] == 'summer']['total_value']
autumn_data = sales_data[sales_data['season'] == 'autumn']['total_value']

# Ensure there are enough data points in each season
print(f"Spring data count: {len(spring_data)}")
print(f"Summer data count: {len(summer_data)}")
print(f"Autumn data count: {len(autumn_data)}")
print(f"Winter data count: {len(winter_data)}")

# Calculate the mean values for sanity check
mean_values = sales_data.groupby('season')['total_value'].agg(['mean', 'median', 'max', 'sum', 'count'])
print(mean_values)

# Function to perform Kolmogorov-Smirnov test
def ks_test(data, season):
    stat, p_value = stats.kstest(data, 'norm', args=(data.mean(), data.std()))
    print(f"Kolmogorov-Smirnov test for {season}:")
    print(f"  Test statistic: {stat:.6f}")
    print(f"  p-value: {p_value:.20f}")
    if p_value > 0.05:
        print(f"  {season} data is normally distributed.\n")
    else:
        print(f"  {season} data is not normally distributed.\n")

# Perform Kolmogorov-Smirnov test for each season
ks_test(winter_data, 'Winter')
ks_test(spring_data, 'Spring')
ks_test(summer_data, 'Summer')
ks_test(autumn_data, 'Autumn')


In [ ]:
# Perform Kruskal-Wallis Test
kruskal_test = stats.kruskal(winter_data, spring_data, summer_data, autumn_data)

# Display the results
print(f"Kruskal-Wallis test statistic: {kruskal_test.statistic:.6f}")
print(f"Kruskal-Wallis test p-value: {kruskal_test.pvalue:.30f}")

if kruskal_test.pvalue < 0.05:
    print("There are significant variations in profitability across the seasons.")
else:
    print("There are no significant variations in profitability across the seasons.")


In [ ]:
final_data = sales_data[['total_value', 'season']]

In [ ]:
tukey_oneway = pairwise_tukeyhsd(endog = final_data["total_value"], groups = final_data["season"])

# Display the results
tukey_oneway.summary()

In [ ]:
# Extract data for each season
seasons = ['winter', 'spring', 'summer', 'autumn']
data = {season: sales_data[sales_data['season'] == season]['total_value'] for season in seasons}

# Combine the data into a single DataFrame
final_data = pd.concat([pd.DataFrame({'season': season, 'total_value': values}) for season, values in data.items()])

# Perform Dunn's post hoc test
dunn_test = sp.posthoc_dunn(final_data, val_col='total_value', group_col='season', p_adjust='bonferroni')
print(dunn_test)

# Calculate mean ranks
mean_ranks = final_data.groupby('season')['total_value'].mean().sort_values()
print(mean_ranks)

# Annotate directions on the Dunn's test results
direction_matrix = pd.DataFrame(index=seasons, columns=seasons)
for i in seasons:
    for j in seasons:
        if dunn_test.loc[i, j] < 0.05:  # Significant difference
            if mean_ranks[i] > mean_ranks[j]:
                direction_matrix.loc[i, j] = f"{i} > {j}"
            else:
                direction_matrix.loc[i, j] = f"{i} < {j}"
        else:
            direction_matrix.loc[i, j] = 'No significant difference'

print("Direction of Differences:")
print(direction_matrix)


In [ ]:
sales_data.head()

# H1b

In [ ]:
# Extract data for each weekday
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
data = {day: sales_data[sales_data['day_of_week_name'] == day]['total_value'] for day in weekdays}

# Combine the data into a single DataFrame
final_data = pd.concat([pd.DataFrame({'day_of_week_name': day, 'total_value': values}) for day, values in data.items()])

# Perform Kruskal-Wallis Test
kruskal_test = stats.kruskal(*data.values())
print(f"Kruskal-Wallis test statistic: {kruskal_test.statistic:.6f}")
print(f"Kruskal-Wallis test p-value: {kruskal_test.pvalue:.30f}")

if kruskal_test.pvalue < 0.05:
    print("There are significant variations in profitability across the weekdays.")
    
    # Perform Dunn's post hoc test
    dunn_test = sp.posthoc_dunn(final_data, val_col='total_value', group_col='day_of_week_name', p_adjust='bonferroni')
    print(dunn_test)

    # Calculate mean ranks
    mean_ranks = final_data.groupby('day_of_week_name')['total_value'].mean().sort_values()
    print(mean_ranks)

    # Annotate directions on the Dunn's test results
    direction_matrix = pd.DataFrame(index=weekdays, columns=weekdays)
    for i in weekdays:
        for j in weekdays:
            if dunn_test.loc[i, j] < 0.05:  # Significant difference
                if mean_ranks[i] > mean_ranks[j]:
                    direction_matrix.loc[i, j] = f"{i} > {j}"
                else:
                    direction_matrix.loc[i, j] = f"{i} < {j}"
            else:
                direction_matrix.loc[i, j] = 'No significant difference'

    print("Direction of Differences:")
    print(direction_matrix)
else:
    print("There are no significant variations in profitability across the weekdays.")

# H1c

In [ ]:
# Filter the data for the year 2023
sales_data_2023 = sales_data[sales_data['year_integer'] == 2023]

# Extract data for each month in 2023
months = sales_data_2023['year_month'].unique()
data = {month: sales_data_2023[sales_data_2023['year_month'] == month]['total_value'] for month in months}

# Combine the data into a single DataFrame
final_data = pd.concat([pd.DataFrame({'year_month': month, 'total_value': values}) for month, values in data.items()])

# Perform Kruskal-Wallis Test
kruskal_test = stats.kruskal(*data.values())
print(f"Kruskal-Wallis test statistic: {kruskal_test.statistic:.6f}")
print(f"Kruskal-Wallis test p-value: {kruskal_test.pvalue:.30f}")

if kruskal_test.pvalue < 0.05:
    print("There are significant variations in profitability across the months in 2023.")
    
    # Perform Dunn's post hoc test
    dunn_test = sp.posthoc_dunn(final_data, val_col='total_value', group_col='year_month', p_adjust='bonferroni')
    print(dunn_test)

    # Calculate mean ranks
    mean_ranks = final_data.groupby('year_month')['total_value'].mean().sort_values()
    print(mean_ranks)

    # Annotate directions on the Dunn's test results
    direction_matrix = pd.DataFrame(index=months, columns=months)
    for i in months:
        for j in months:
            if dunn_test.loc[i, j] < 0.05:  # Significant difference
                if mean_ranks[i] > mean_ranks[j]:
                    direction_matrix.loc[i, j] = f"{i} > {j}"
                else:
                    direction_matrix.loc[i, j] = f"{i} < {j}"
            else:
                direction_matrix.loc[i, j] = 'No significant difference'

    print("Direction of Differences:")
    print(direction_matrix)
else:
    print("There are no significant variations in profitability across the months in 2023.")

# Hypotheses 2

## Support, Confidence and Lift show Seasonal Patterns 

In [ ]:
# Ensure the 'trx_date' column is in datetime format
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Function to assign seasons
def assign_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'autumn'

# Assign seasons
sales_data['season'] = sales_data['trx_date'].dt.month.apply(assign_season)

# Initialize lists to hold all the results
all_confidences = []
all_supports = []
all_lifts = []

# Loop through each shop and each season to generate association rules
for shop in sales_data['shopID'].unique():
    for year in sales_data['year_integer'].unique():
        for season in ['spring', 'summer', 'autumn', 'winter']:
            # Filter data for the specific shop, year, and season
            shop_season_data = sales_data[(sales_data['shopID'] == shop) & 
                                          (sales_data['year_integer'] == year) & 
                                          (sales_data['season'] == season)]
            
            # If there's not enough data, skip
            if shop_season_data.empty:
                continue
            
            # Create a basket dataframe
            basket = shop_season_data.groupby(['trx_id', 'product_code'])['quantity'].sum().unstack().fillna(0)
            basket = basket.map(lambda x: x > 0).astype(bool)  # Convert to boolean matrix
            
            # Generate frequent itemsets
            frequent_itemsets = apriori(basket, min_support=0.001, use_colnames=True)
            
            # Generate the association rules
            rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)
            
            # Add the mean confidence, support, and lift for this shop and season to the lists
            if not rules.empty:
                mean_confidence = rules['confidence'].mean()
                mean_support = rules['support'].mean()
                mean_lift = rules['lift'].mean()
                all_confidences.append({'shop': shop, 'year': year, 'season': season, 'mean_confidence': mean_confidence})
                all_supports.append({'shop': shop, 'year': year, 'season': season, 'mean_support': mean_support})
                all_lifts.append({'shop': shop, 'year': year, 'season': season, 'mean_lift': mean_lift})

# Convert the lists to DataFrames
confidences_df = pd.DataFrame(all_confidences)
supports_df = pd.DataFrame(all_supports)
lifts_df = pd.DataFrame(all_lifts)

# Display the seasonal mean confidence, support, and lift values
print("Confidence DataFrame")
print(confidences_df)
print("Support DataFrame")
print(supports_df)
print("Lift DataFrame")
print(lifts_df)

# Function to perform Kolmogorov-Smirnov test
def ks_test(data, metric, season):
    stat, p_value = stats.kstest(data, 'norm', args=(data.mean(), data.std()))
    print(f"Kolmogorov-Smirnov test for {metric} in {season}:")
    print(f"  Test statistic: {stat:.6f}")
    print(f"  p-value: {p_value:.6f}")
    if p_value > 0.05:
        print(f"  {metric} in {season} data is normally distributed.\n")
    else:
        print(f"  {metric} in {season} data is not normally distributed.\n")

# Function to check normality for each metric and season
def check_normality(df, metric):
    for season in ['spring', 'summer', 'autumn', 'winter']:
        season_data = df[df['season'] == season][metric]
        if len(season_data) > 0:
            ks_test(season_data, metric, season)

# Check normality for mean_confidence, mean_support, and mean_lift
print("Checking normality for mean_confidence")
check_normality(confidences_df, 'mean_confidence')
print("Checking normality for mean_support")
check_normality(supports_df, 'mean_support')
print("Checking normality for mean_lift")
check_normality(lifts_df, 'mean_lift')

# Perform Kruskal-Wallis test to see if there's a significant difference between the seasonal mean values
def perform_kruskal_wallis_test(metric, df):
    spring_values = df[df['season'] == 'spring'][metric]
    summer_values = df[df['season'] == 'summer'][metric]
    autumn_values = df[df['season'] == 'autumn'][metric]
    winter_values = df[df['season'] == 'winter'][metric]
    
    kruskal_test = stats.kruskal(spring_values, summer_values, autumn_values, winter_values)
    
    print(f"Kruskal-Wallis test statistic for {metric}: {kruskal_test.statistic:.6f}")
    print(f"Kruskal-Wallis test p-value for {metric}: {kruskal_test.pvalue:.30f}")
    
    if kruskal_test.pvalue < 0.05:
        print(f"There are significant differences in mean {metric} values across the seasons.")
    else:
        print(f"There are no significant differences in mean {metric} values across the seasons.")

# Perform tests for confidence, support, and lift
perform_kruskal_wallis_test('mean_confidence', confidences_df)
perform_kruskal_wallis_test('mean_support', supports_df)
perform_kruskal_wallis_test('mean_lift', lifts_df)


In [ ]:
# Perform Dunn's post hoc test for lift values
dunn_test = sp.posthoc_dunn(lifts_df, val_col='mean_lift', group_col='season', p_adjust='bonferroni')
print("Dunn's test results for lift values across seasons:")
print(dunn_test)

# Calculate mean ranks
mean_ranks = lifts_df.groupby('season')['mean_lift'].mean().sort_values()
print("Mean ranks for lift values:")
print(mean_ranks)

# Annotate directions on the Dunn's test results
seasons = ['spring', 'summer', 'autumn', 'winter']
direction_matrix = pd.DataFrame(index=seasons, columns=seasons)
for i in seasons:
    for j in seasons:
        if dunn_test.loc[i, j] < 0.05:  # Significant difference
            if mean_ranks[i] > mean_ranks[j]:
                direction_matrix.loc[i, j] = f"{i} > {j}"
            else:
                direction_matrix.loc[i, j] = f"{i} < {j}"
        else:
            direction_matrix.loc[i, j] = 'No significant difference'

print("Direction of Differences:")
print(direction_matrix)

In [ ]:
# Perform Dunn's post hoc test for support values
dunn_test_support = sp.posthoc_dunn(supports_df, val_col='mean_support', group_col='season', p_adjust='bonferroni')
print("Dunn's test results for support values across seasons:")
print(dunn_test_support)

# Calculate mean ranks for support values
mean_ranks_support = supports_df.groupby('season')['mean_support'].mean().sort_values()
print("Mean ranks for support values:")
print(mean_ranks_support)

# Annotate directions on the Dunn's test results for support values
seasons = ['spring', 'summer', 'autumn', 'winter']
direction_matrix_support = pd.DataFrame(index=seasons, columns=seasons)
for i in seasons:
    for j in seasons:
        if dunn_test_support.loc[i, j] < 0.05:  # Significant difference
            if mean_ranks_support[i] > mean_ranks_support[j]:
                direction_matrix_support.loc[i, j] = f"{i} > {j}"
            else:
                direction_matrix_support.loc[i, j] = f"{i} < {j}"
        else:
            direction_matrix_support.loc[i, j] = 'No significant difference'

print("Direction of Differences for Support Values:")
print(direction_matrix_support)


In [ ]:
# Ensure the 'trx_date' column is in datetime format
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Assign seasons
def assign_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'autumn'

sales_data['season'] = sales_data['trx_date'].dt.month.apply(assign_season)

# Filter data for winter and summer
winter_data = sales_data[sales_data['season'] == 'winter']
summer_data = sales_data[sales_data['season'] == 'summer']

# Calculate average basket size for winter
winter_basket_size = winter_data.groupby('trx_id')['product_code'].count().mean()

# Calculate average basket size for summer
summer_basket_size = summer_data.groupby('trx_id')['product_code'].count().mean()

print(f"Average basket size in winter: {winter_basket_size}")
print(f"Average basket size in summer: {summer_basket_size}")

In [ ]:
# Assuming sales_data is already loaded as a DataFrame

# Ensure the 'trx_date' column is in datetime format
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Assign seasons
def assign_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'autumn'

sales_data['season'] = sales_data['trx_date'].dt.month.apply(assign_season)

# Group by season and count the unique product codes
products_per_season = sales_data.groupby('season')['product_code'].nunique()

print("Number of different products sold per season:")
print(products_per_season)

In [ ]:
import pandas as pd
from collections import Counter

# Load your sales data into a DataFrame (assuming it's already done)
# sales_data = pd.read_csv('your_sales_data.csv')

# Filter transactions that include Product ID_44
product_id = 'ID_44'
transactions_with_product = sales_data[sales_data['product_code'] == product_id]

# Get the transaction IDs for transactions that include Product ID_44
transaction_ids = transactions_with_product['trx_id'].unique()

# Filter the sales data to include only these transactions
filtered_sales_data = sales_data[sales_data['trx_id'].isin(transaction_ids)]

# Count the frequency of other products bought with Product ID_44
product_counts = Counter(filtered_sales_data['product_code'])

# Count the number of transactions where Product ID_44 was bought alone
transactions_with_product_alone = transactions_with_product['trx_id'].isin(
    filtered_sales_data.groupby('trx_id').filter(lambda x: len(x) == 1)['trx_id']
).sum()

# Remove Product ID_44 from the count for other products
del product_counts[product_id]

# Convert the counter to a DataFrame for easy viewing
product_counts_df = pd.DataFrame(product_counts.items(), columns=['Product_Code', 'Count'])
product_counts_df = product_counts_df.sort_values(by='Count', ascending=False)

# Display the top 10 products bought with Product ID_44
print("Product ID_44 bought alone:", transactions_with_product_alone)
print(product_counts_df.head(10))


In [ ]:
import pandas as pd
from collections import Counter

# Ensure the 'trx_date' column is in datetime format
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Filter transactions from the year 2020
sales_data_2020 = sales_data[sales_data['trx_date'].dt.year == 2020]

# Filter transactions that include Product ID_44
product_id = 'ID_60'
transactions_with_product = sales_data_2020[sales_data_2020['product_code'] == product_id]

# Get the transaction IDs for transactions that include Product ID_44
transaction_ids = transactions_with_product['trx_id'].unique()

# Filter the sales data to include only these transactions
filtered_sales_data = sales_data_2020[sales_data_2020['trx_id'].isin(transaction_ids)]

# Count the frequency of other products bought with Product ID_44
product_counts = Counter(filtered_sales_data['product_code'])

# Count the number of transactions where Product ID_44 was bought alone
transactions_with_product_alone = transactions_with_product['trx_id'].isin(
    filtered_sales_data.groupby('trx_id').filter(lambda x: len(x) == 1)['trx_id']
).sum()

# Remove Product ID_44 from the count for other products
del product_counts[product_id]

# Convert the counter to a DataFrame for easy viewing
product_counts_df = pd.DataFrame(product_counts.items(), columns=['Product_Code', 'Count'])
product_counts_df = product_counts_df.sort_values(by='Count', ascending=False)

# Display the top 10 products bought with Product ID_44
print("Product ID_44 bought alone:", transactions_with_product_alone)
print(product_counts_df.head(10))


In [ ]:
import pandas as pd

# Ensure the 'trx_date' column is in datetime format
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Filter transactions from the year 2020
sales_data_2020 = sales_data[sales_data['trx_date'].dt.year == 2020]

# Filter transactions that include Product ID_44 and Product ID_60
product_id_44 = 'ID_44'
product_id_60 = 'ID_60'

transactions_with_product_44 = sales_data_2020[sales_data_2020['product_code'] == product_id_44]
transactions_with_product_60 = sales_data_2020[sales_data_2020['product_code'] == product_id_60]

# Get the transaction IDs for transactions that include Product ID_44 and Product ID_60
transaction_ids_44 = set(transactions_with_product_44['trx_id'].unique())
transaction_ids_60 = set(transactions_with_product_60['trx_id'].unique())

# Find transactions that include both Product ID_44 and Product ID_60
common_transaction_ids = transaction_ids_44.intersection(transaction_ids_60)

# Count the occurrences
total_occurrences_44 = len(transactions_with_product_44)
total_occurrences_60 = len(transactions_with_product_60)
together_count = len(common_transaction_ids)

# Print the results
print(f"Total occurrences of Product ID_44 in 2020: {total_occurrences_44}")
print(f"Total occurrences of Product ID_60 in 2020: {total_occurrences_60}")
print(f"Number of times Product ID_44 and Product ID_60 were bought together in 2020: {together_count}")


# Hypotheses 3

## Shops with similar customer purchase patterns (clustered based on association rules) show similar sales growth rates.

In [ ]:
# Assuming `sales_data` is the DataFrame with the provided structure

# Step 1: Extract Total Annual Sales per Shop
annual_sales = sales_data.groupby(['shopID', 'year_integer'])['total_value'].sum().reset_index()

# Step 2: Calculate Sales Growth Rates for Each Shop
annual_sales['sales_growth'] = annual_sales.groupby('shopID')['total_value'].pct_change()

# Remove the first year for each shop as it will have NaN sales growth
annual_sales = annual_sales.dropna(subset=['sales_growth'])

# Define the clusters
cluster_1 = [1, 3, 5]
cluster_2 = [6, 7, 8, 9]
cluster_3 = [4]
cluster_4 = [11, 12, 13]

# Step 3: Separate the growth rates by cluster
cluster_1_growth = annual_sales[annual_sales['shopID'].isin(cluster_1)]['sales_growth']
cluster_2_growth = annual_sales[annual_sales['shopID'].isin(cluster_2)]['sales_growth']
cluster_3_growth = annual_sales[annual_sales['shopID'].isin(cluster_3)]['sales_growth']
cluster_4_growth = annual_sales[annual_sales['shopID'].isin(cluster_4)]['sales_growth']

# Step 4: Perform ANOVA test to compare the sales growth rates between clusters
anova_result = stats.f_oneway(cluster_1_growth, cluster_2_growth, cluster_3_growth, cluster_4_growth)

print(f"Cluster 1 sales growth rates: {cluster_1_growth.values}")
print(f"Cluster 2 sales growth rates: {cluster_2_growth.values}")
print(f"Cluster 3 sales growth rates: {cluster_3_growth.values}")
print(f"Cluster 4 sales growth rates: {cluster_4_growth.values}")
print(f"ANOVA test statistic: {anova_result.statistic:.6f}")
print(f"ANOVA test p-value: {anova_result.pvalue:.6f}")

if anova_result.pvalue < 0.05:
    print("There are significant differences in sales growth rates between the clusters.")
else:
    print("There are no significant differences in sales growth rates between the clusters.")


In [ ]:
# Ensure the trx_date column is in datetime format
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Extract the year and quarter from the trx_date
sales_data['year_quarter'] = sales_data['trx_date'].dt.to_period('Q')

# Step 1: Extract Total Quarterly Sales per Shop
quarterly_sales = sales_data.groupby(['shopID', 'year_quarter'])['total_value'].sum().reset_index()

# Step 2: Calculate Quarterly Growth Rates for Each Shop
quarterly_sales['quarterly_growth'] = quarterly_sales.groupby('shopID')['total_value'].pct_change()

# Remove the first quarter for each shop as it will have NaN sales growth
quarterly_sales = quarterly_sales.dropna(subset=['quarterly_growth'])

# Define the clusters
cluster_1 = [1, 3, 5]
cluster_2 = [6, 7, 8, 9]
cluster_4 = [11, 12, 13]

# Function to check normality within a cluster
def check_normality_within_cluster(cluster, cluster_name):
    cluster_growth = quarterly_sales[quarterly_sales['shopID'].isin(cluster)]
    normality_results = {}
    for shop in cluster:
        shop_growth = cluster_growth[cluster_growth['shopID'] == shop]['quarterly_growth']
        if len(shop_growth) >= 3:
            result = stats.shapiro(shop_growth)
            normality_results[shop] = result
            print(f"Shop {shop}: W-statistic={result.statistic:.6f}, p-value={result.pvalue:.6f}")
            if result.pvalue < 0.05:
                print(f"Shop {shop}'s growth rates are not normally distributed.")
            else:
                print(f"Shop {shop}'s growth rates are normally distributed.")
        else:
            print(f"Shop {shop} does not have enough data points for normality test.")
            normality_results[shop] = None
    print()
    
    return normality_results

# Function to perform ANOVA test within a cluster
def perform_anova_within_cluster(cluster, cluster_name):
    cluster_growth = quarterly_sales[quarterly_sales['shopID'].isin(cluster)]
    anova_result = stats.f_oneway(
        *[cluster_growth[cluster_growth['shopID'] == shop]['quarterly_growth'] for shop in cluster]
    )
    
    print(f"ANOVA test results for {cluster_name}:")
    print(f"ANOVA test statistic: {anova_result.statistic:.6f}")
    print(f"ANOVA test p-value: {anova_result.pvalue:.6f}")
    
    if anova_result.pvalue < 0.05:
        print(f"There are significant differences in quarterly growth rates within {cluster_name}.")
    else:
        print(f"There are no significant differences in quarterly growth rates within {cluster_name}.")
    print()

# Check normality and perform ANOVA test within each cluster
normality_results_cluster_1 = check_normality_within_cluster(cluster_1, 'Cluster 1')
normality_results_cluster_2 = check_normality_within_cluster(cluster_2, 'Cluster 2')
normality_results_cluster_4 = check_normality_within_cluster(cluster_4, 'Cluster 4')

perform_anova_within_cluster(cluster_1, 'Cluster 1')
perform_anova_within_cluster(cluster_2, 'Cluster 2')
perform_anova_within_cluster(cluster_4, 'Cluster 4')


In [ ]:
# Ensure the trx_date column is in datetime format
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Extract the year and quarter from the trx_date
sales_data['year_quarter'] = sales_data['trx_date'].dt.to_period('Q')

# Step 1: Extract Total Quarterly Sales per Shop
quarterly_sales = sales_data.groupby(['shopID', 'year_quarter'])['total_value'].sum().reset_index()

# Step 2: Calculate Quarterly Growth Rates for Each Shop
quarterly_sales['quarterly_growth'] = quarterly_sales.groupby('shopID')['total_value'].pct_change()

# Remove the first quarter for each shop as it will have NaN sales growth
quarterly_sales = quarterly_sales.dropna(subset=['quarterly_growth'])

# Define the clusters
cluster_1 = [1, 3, 5]
cluster_2 = [6, 7, 8, 9]
cluster_4 = [11, 12, 13]

# Function to perform Kruskal-Wallis test within a cluster
def perform_kruskal_wallis_within_cluster(cluster, cluster_name):
    cluster_growth = quarterly_sales[quarterly_sales['shopID'].isin(cluster)]
    kruskal_result = stats.kruskal(
        *[cluster_growth[cluster_growth['shopID'] == shop]['quarterly_growth'] for shop in cluster]
    )
    
    print(f"Kruskal-Wallis test results for {cluster_name}:")
    print(f"Kruskal-Wallis test statistic: {kruskal_result.statistic:.6f}")
    print(f"Kruskal-Wallis test p-value: {kruskal_result.pvalue:.6f}")
    
    if kruskal_result.pvalue < 0.05:
        print(f"There are significant differences in quarterly growth rates within {cluster_name}.")
    else:
        print(f"There are no significant differences in quarterly growth rates within {cluster_name}.")
    print()

# Perform Kruskal-Wallis test within each cluster
perform_kruskal_wallis_within_cluster(cluster_1, 'Cluster 1')
perform_kruskal_wallis_within_cluster(cluster_2, 'Cluster 2')
perform_kruskal_wallis_within_cluster(cluster_4, 'Cluster 4')


In [ ]:
# Ensure the trx_date column is in datetime format
sales_data['trx_date'] = pd.to_datetime(sales_data['trx_date'])

# Extract the year and quarter from the trx_date
sales_data['year_quarter'] = sales_data['trx_date'].dt.to_period('Q')

# Step 1: Extract Total Quarterly Sales per Shop
quarterly_sales = sales_data.groupby(['shopID', 'year_quarter'])['total_value'].sum().reset_index()

# Step 2: Calculate Quarterly Growth Rates for Each Shop
quarterly_sales['quarterly_growth'] = quarterly_sales.groupby('shopID')['total_value'].pct_change()

# Remove the first quarter for each shop as it will have NaN sales growth
quarterly_sales = quarterly_sales.dropna(subset=['quarterly_growth'])

# Define the clusters
cluster_1 = [1, 3, 5]
cluster_2 = [6, 7, 8, 9]
cluster_4 = [11, 12, 13]

# Function to calculate Coefficient of Variation (CV) for a cluster
def calculate_cv(cluster, cluster_name):
    cluster_growth = quarterly_sales[quarterly_sales['shopID'].isin(cluster)]
    cv = cluster_growth.groupby('shopID')['quarterly_growth'].std() / cluster_growth.groupby('shopID')['quarterly_growth'].mean()
    print(f"Coefficient of Variation (CV) for {cluster_name}:")
    print(cv)
    print(f"Mean CV for {cluster_name}: {cv.mean()}")
    print()


# Perform CV within each cluster
calculate_cv(cluster_1, 'Cluster 1')
calculate_cv(cluster_2, 'Cluster 2')
calculate_cv(cluster_4, 'Cluster 4')

# Recommendation Algorithm Code

In [ ]:
# Function to extract product codes from frozenset data types in antecedents and consequents
def extract_product_codes(antecedents, consequents):
    return list(antecedents.union(consequents))

# Function to create a rule representation from antecedents and consequents
def create_rule_string(antecedents, consequents):
    return f"{antecedents} -> {consequents}"

# Function to normalize a list of metrics
def normalize_metric(metric):
    min_val = min(metric)
    max_val = max(metric)
    return [(m - min_val) / (max_val - min_val) for m in metric]

# Function to recommend products based on given products
def recommend_products(meta_dfs_redundant, input_products, total_metric_threshold=2, lift_weight=2):
    recommendations = defaultdict(list)
    
    for shop, years_data in meta_dfs_redundant.items():
        for year, df in years_data.items():
            for _, row in df.iterrows():
                antecedents = row['antecedents']
                if antecedents.issubset(input_products):
                    recommendations[row['consequents']].append((row['lift'], row['confidence'], row['support'], row['conviction']))

    if not recommendations:
        print("No recommendations found.")
        return []

    # Flatten the recommendations
    recommendations_list = []
    for consequent, metrics in recommendations.items():
        avg_metrics = tuple(map(lambda x: sum(x) / len(x), zip(*metrics)))
        recommendations_list.append((consequent, avg_metrics[0], avg_metrics[1], avg_metrics[2], avg_metrics[3]))

    # Normalize metrics
    lifts = [rec[1] for rec in recommendations_list]
    confidences = [rec[2] for rec in recommendations_list]
    supports = [rec[3] for rec in recommendations_list]
    convictions = [rec[4] for rec in recommendations_list]

    normalized_lifts = normalize_metric(lifts)
    normalized_confidences = normalize_metric(confidences)
    normalized_supports = normalize_metric(supports)
    normalized_convictions = normalize_metric(convictions)

    # Calculate the total metric and update recommendations_list
    total_metrics = [
        normalized_lifts[i] * lift_weight + normalized_confidences[i] + normalized_supports[i] + normalized_convictions[i]
        for i in range(len(recommendations_list))
    ]
    for i in range(len(recommendations_list)):
        recommendations_list[i] = recommendations_list[i] + (total_metrics[i],)

    # Sort by the total metric
    recommendations_list.sort(key=lambda x: x[5], reverse=True)

    if not recommendations_list:
        print("All recommendations were excluded.")
        return []

    # Exclude items that are part of the input
    recommendations_list = [rec for rec in recommendations_list if not any(item in input_products for item in rec[0])]

    if not recommendations_list:
        print("All recommendations were excluded after filtering input products.")
        return []

    # Filter out recommendations with total_metric less than 1
    recommendations_list = [rec for rec in recommendations_list if rec[5] >= 1]

    if not recommendations_list:
        print("All recommendations were excluded due to low total_metric.")
        return []

    if len(recommendations_list) < 5:
        return [(rec[0], rec[5]) for rec in recommendations_list]

    # Check if the step between the first and subsequent recommendations is not very high using total metric values
    results = [(recommendations_list[0][0], recommendations_list[0][5])]
    for i in range(1, min(5, len(recommendations_list))):
        current_total_metric = recommendations_list[i][5]
        previous_total_metric = recommendations_list[i - 1][5]
        if (previous_total_metric - current_total_metric) / previous_total_metric > total_metric_threshold:
            break
        results.append((recommendations_list[i][0], current_total_metric))

    return results


In [ ]:
# Group the data by trx_id and count the number of products in each transaction
products_per_transaction = sales_data.groupby('trx_id').size()

# Calculate the average number of products per transaction
average_products_per_transaction = products_per_transaction.mean()

print(f"Average number of products per transaction: {average_products_per_transaction:.2f}")


In [ ]:
# Filter the data for the year 2023
sales_data_2023 = sales_data[sales_data['year_integer'] == 2023]

# Group the data by trx_id and count the number of products in each transaction for the year 2023
products_per_transaction_2023 = sales_data_2023.groupby('trx_id').size()

# Calculate the average number of products per transaction for the year 2023
average_products_per_transaction_2023 = products_per_transaction_2023.mean()

print(f"Average number of products per transaction in 2023: {average_products_per_transaction_2023:.2f}")


# Recommendation Algorithm Test

In [ ]:
# Example input: single product
input_products = {'ID_44'}

# Call the function
recommendations = recommend_products(meta_dfs_redundant, input_products)

# Print recommendations with paragraph breaks
print("\n\n".join(f"Recommended product: {', '.join(product)}, Total Metric: {total_metric:.2f}" for product, total_metric in recommendations))

# Extract product IDs and their respective total metric values for plotting
products = [', '.join(list(rec[0])) for rec in recommendations][:5]
total_metrics = [rec[1] for rec in recommendations][:5]

# Plot the horizontal bar graph
plt.figure(figsize=(12, 6))
bars = plt.barh(products, total_metrics, color=plt.cm.Reds(np.array(total_metrics) / max(total_metrics)))

# Annotate the bars with product names and total_metric values
for bar, product, metric in zip(bars, products, total_metrics):
    plt.text(bar.get_width()/2, bar.get_y() + bar.get_height()/2,
             f"{product}, Strength: {metric:.2f}", ha='center', va='center', fontsize=10, color='white')

# Remove x-axis and y-axis labels, ticks, and the coordinate system
plt.gca().xaxis.set_visible(False)
plt.gca().yaxis.set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

# Reverse the order of the products on the y-axis
plt.gca().invert_yaxis()

# Save the plot as an SVG file
plt.savefig("recommended_products1.svg", format='svg')

plt.show()


In [ ]:
# Example input: single product
input_products = {'ID_8', 'ID_16', 'ID_9'}
# Call the function
recommendations = recommend_products(meta_dfs_redundant, input_products)

# Print recommendations with paragraph breaks
print("\n\n".join(f"Recommended product: {', '.join(product)}, Total Metric: {total_metric:.2f}" for product, total_metric in recommendations))

# Extract product IDs and their respective total metric values for plotting
products = [', '.join(list(rec[0])) for rec in recommendations][:5]
total_metrics = [rec[1] for rec in recommendations][:5]

# Plot the horizontal bar graph
plt.figure(figsize=(12, 6))
bars = plt.barh(products, total_metrics, color=plt.cm.Reds(np.array(total_metrics) / max(total_metrics)))

# Annotate the bars with product names and total_metric values
for bar, product, metric in zip(bars, products, total_metrics):
    plt.text(bar.get_width()/2, bar.get_y() + bar.get_height()/2,
             f"{product}, Strength: {metric:.2f}", ha='center', va='center', fontsize=10, color='white')

# Remove x-axis and y-axis labels, ticks, and the coordinate system
plt.gca().xaxis.set_visible(False)
plt.gca().yaxis.set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

# Reverse the order of the products on the y-axis
plt.gca().invert_yaxis()

# Save the plot as an SVG file
plt.savefig("recommended_products2.svg", format='svg')

plt.show()


In [ ]:
# Example input: single product
input_products = {'ID_27','ID_400', 'ID_95'}

# Call the function
recommendations = recommend_products(meta_dfs_redundant, input_products)

# Print recommendations with paragraph breaks
print("\n\n".join(f"Recommended product: {', '.join(product)}, Total Metric: {total_metric:.2f}" for product, total_metric in recommendations))

# Extract product IDs and their respective total metric values for plotting
products = [', '.join(list(rec[0])) for rec in recommendations][:5]
total_metrics = [rec[1] for rec in recommendations][:5]

# Plot the horizontal bar graph
plt.figure(figsize=(12, 6))
bars = plt.barh(products, total_metrics, color=plt.cm.Reds(np.array(total_metrics) / max(total_metrics)))

# Annotate the bars with product names and total_metric values
for bar, product, metric in zip(bars, products, total_metrics):
    plt.text(bar.get_width()/2, bar.get_y() + bar.get_height()/2,
             f"{product}, Strength: {metric:.2f}", ha='center', va='center', fontsize=10, color='white')

# Remove x-axis and y-axis labels, ticks, and the coordinate system
plt.gca().xaxis.set_visible(False)
plt.gca().yaxis.set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

# Reverse the order of the products on the y-axis
plt.gca().invert_yaxis()

# Save the plot as an SVG file
plt.savefig("recommended_products3.svg", format='svg')

plt.show()


In [ ]:
# Example input: single product
input_products = {'ID_64', 'ID_32'}

# Call the function
recommendations = recommend_products(meta_dfs_redundant, input_products)

# Print recommendations with paragraph breaks
print("\n\n".join(f"Recommended product: {', '.join(product)}, Total Metric: {total_metric:.2f}" for product, total_metric in recommendations))

# Extract product IDs and their respective total metric values for plotting
products = [', '.join(list(rec[0])) for rec in recommendations][:5]
total_metrics = [rec[1] for rec in recommendations][:5]

# Plot the horizontal bar graph
plt.figure(figsize=(12, 6))
bars = plt.barh(products, total_metrics, color=plt.cm.Reds(np.array(total_metrics) / max(total_metrics)))

# Annotate the bars with product names and total_metric values
for bar, product, metric in zip(bars, products, total_metrics):
    plt.text(bar.get_width()/2, bar.get_y() + bar.get_height()/2,
             f"{product}, Strength: {metric:.2f}", ha='center', va='center', fontsize=10, color='white')

# Remove x-axis and y-axis labels, ticks, and the coordinate system
plt.gca().xaxis.set_visible(False)
plt.gca().yaxis.set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)

# Reverse the order of the products on the y-axis
plt.gca().invert_yaxis()

# Save the plot as an SVG file
plt.savefig("recommended_products3.svg", format='svg')

plt.show()
